In [ ]:
import pandas as  pd
import numpy as np
import sys
from pathlib import Path
from glob import glob
import dateutil.parser
import collections
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import json

#### Read Redcap data into Pandas dataframe

The Redcap data were stored in different projects, and exported in different formats. 
This section reads all the RedCap data into a single Pandas dataframe that forms the 
basis of all further analysis.

In [ ]:
redcap = Path("../../../../../BANDA215/Redcap2022")
df_male = pd.read_table(redcap / "BANDAAdolescentMale_DATA_2022-01-26_1933.csv")
df_female = pd.read_csv(redcap / "BANDAAdolescentFemal_DATA_2022-01-26_1932.csv")
df_parents = pd.read_table(redcap / "BANDAParent_DATA_2022-01-26_1933.csv")
df_ids = pd.read_table(redcap / "BANDAData_DATA_2022-01-26_1936.csv")
df_clinics = pd.read_table(redcap / "BANDAClinicalIntervi_DATA_2022-01-26_1934.csv",
                          na_values=["n/a"])

df_male.columns = [var if "subject" not in var else "subject_id" for var in df_male.columns]
df_parents.columns = [var if "subject" not in var else "subject_id" for var in df_parents.columns]
df_parents.drop(df_parents[df_parents.parent_initial_timestamp == '[not completed]'].index, inplace=True)

df_all = pd.concat([df_male, df_female], axis=0, ignore_index=True)

subjmap = {}
df_subjmap = pd.read_csv("nda-subject-map.csv")
for val in df_subjmap.iterrows():
    subjmap[val[1].src_subject_id] = val[1].subjectkey
subjmap_rev = {v:k for k,v in subjmap.items()}

# Map NDA Pseudo GUIDS to local subject identifiers
old_subj_info = pd.read_csv("../../../../C3037 - Mappings/c3037_ndar_subject_info.csv")

In [ ]:
# Redcap stores different timepoints of a longitudinal survey in variables with appended suffixes
suffixmap = {"_i": "T1", "_1": "T2", "_2": "T3", "_3": "T4"}
# map visits to keys
timemap = {"_initial_": "T1", "_6_": "T2", "_12_": "T3", "_18_": "T4"}
nda_vars = ['src_subject_id', 'subject_id', 'visit', 'interview_date']

def get_timemap(df, regex):
    """Given a regex this function uses relevant timestamp columns to map to visits"""
    timestamps = [var for var in df.columns if re.match(f"{regex}timestamp$", var)]
    #print(timestamps)
    ts2period = {}
    for loc in timestamps:
        for point in timemap:
            if point in loc:
                ts2period[loc] = timemap[point]
                continue
    ts2period_rev = {v:k for k,v in ts2period.items()}
    suffixmap_ts = {k:ts2period_rev[v] for k, v in suffixmap.items() if v in ts2period_rev}
    suffixmap_filt = {k:v for k, v in suffixmap.items() if v in ts2period_rev}
    return ts2period, ts2period_rev, suffixmap_filt, suffixmap_ts

#### Create dictionary of mappings from Redcap variables to NDA data elements

The mappings file also encodes transformations necessary to convert BANDA Redcap
variables to the corresponding NDA data elements. These mappings are provided
in the crosswalk file from the NDA study downloads.

In [ ]:
df_maps = []
var_map = {}
assess_map = {}
for fname in glob("../../../../C3037 - Mappings/*mappings.csv"):
    df_map = pd.read_csv(fname)
    print(Path(fname).name, df_map[["assessment", "ndar_structure"]].drop_duplicates().values.tolist())
    df_maps.append(df_map)
    for var in df_map.banda_var.unique():
        var_map[var] = len(df_maps) - 1
    for assess in df_map.ndar_structure.dropna().unique():
        assess_map[assess] = len(df_maps) - 1
    if 'na_val' in df_map.columns:
        print("Missing values: ", df_map.na_val.unique().tolist())

In [ ]:
pd.concat(df_maps, axis=0).to_csv("Crosswalk.csv", index=False)

#### Transformations to apply to the data

The original intent was to be able to reverse every transformation. Given the 
complications of transforming data from Redcap to NDA, only the forward 
transformations are relevant. The function names correspond to the transform
column in the mapping file.

In [ ]:
def bisbas_rev(x, invert=False): 
    if x not in range(1, 5):
        raise ValueError(f"{x} not in [1, 4]")
    recode = {v: 5-v for v in range(1, 5)}
    return recode[x]

def handed(x, invert=False):
    if not invert:
        if x == 1: return 0
        if x == 3: return 1
    else:
        if x == 1: return 3
        if x == 0: return 1
    return x

def plus1(x, invert=False):
    return x - 1 if invert else x + 1

def plus1_rev(x, invert=False):
    if not invert:
        x = x + 1
    recode = {v: 5-v for v in range(1, 5)}
    x = recode[x]
    if invert:
        x = x - 1
    return x

def minus1(x, invert=False):
    return x + 1 if invert else x - 1

def marital(x, invert=False):
    remap = {0:8, 1:6, 2:5, 3:4, 4:7}
    if invert:
        remap_rev = {v:k for k,v in remap.items()}
        return remap_rev[x] if x in remap_rev else x
    else:
        return remap[x] if x in remap else x

def TOSTRING(x, invert=False):
    if invert:
        raise Exception("Cannot invert")
    masqmap = {1: "not at all",
             2: "a little bit",
             3: "moderately",
             4: "quite a bit",
             5: "extremely"}
    return masqmap[x]

def REVERSE(x, invert=False):
    if x not in range(1, 6):
        raise ValueError(f"{x} not in [1, 6]")
    recode = {x: 6-x for x in range(1, 6)}
    return recode[x]

def rmbi_rev(x, invert=False):
    if x not in range(4):
        ValueError(f"{x} not in [0, 3]")
    recode = {x: 3-x for x in range(4)}
    return recode[x]

def cssrs(x, invert=False):
    if invert:
        if x == 2: return 0
    else:
        if x == 0: return 2
    return x

def age(x, invert=False):
    if invert:
        return x/12
    return x*12

def fhs(x, invert=False):
    if invert:
        raise Exception("Cannot invert")
    if x == 9:
        return np.nan
    return x
    
def ksads(x, invert=False):
    if x == 3: return 4
    if x == 4: return 3
    return x

def map_race(x, invert=False):
    racemap = {0: "American Indian/Alaska Native",
               1: "Asian",
               2: "Hawaiian or Pacific Islander",
               3: "Black or African American", 
               4: "White", 
               5: "More than one race", 
               6: "Unknown or not reported"}
    racemap_rev = {v:k for k,v in racemap.items()}
    if invert:
        raise Exception("Cannot invert")
    else:
        races = []
        for val in np.where(x)[0].tolist():
            races.append(racemap[val])
        if len(races) == 0:
            return "Unknown or not reported"
        if len(races) > 1:
            return "More than one race"
        return races.pop()

def map_sex(x, invert=False):
    sexmap = {0: "Male",
              1: "Female"}
    sexmap_rev = {v:k for k,v in sexmap.items()}
    if invert:
        return sexmap_rev[x]
    else:
        return sexmap[x]

def map_ethnicity(x, invert=False):
    ethnicmap = {0: "Not Hispanic or Latino",
                 1: "Hispanic or Latino"}
    ethnicmap_rev = {v:k for k,v in ethnicmap.items()}
    if invert:
        return ethnicmap_rev[x]
    else:
        return ethnicmap[x]

def map_education(x, invert=False):
    edumap = {0: "8th Grade or Less",
              1: "Some High School",
              2: "Finished High School",
              3: "Completed GED",
              4: "Vocation/Trade/Business School",
              5: "Some College or 2 year degree",
              6: "Finished 4 year degree",
              7: "Master's degree or equivalent",
              8: "Other Advanced degree",
              9: "Unknown"}
    edumap_rev = {v:k for k,v in edumap.items()}
    if invert:
        return edumap_rev[x]
    else:
        return edumap[x]

def map_income(x, invert=False):
    incmap = {0: 3, #"Gross earnings (self)"
              1: 8, #"Other income"
              2: 8, #"Other income"
              3: 2, #"Disability income"
              4: 8, #"Other income"
             }
    if invert: 
        raise Exception("Cannot invert")
    incomes = []
    for val in np.where(x)[0].tolist():
        incomes.append(incmap[val])
    if len(incomes) == 0:
        return np.nan
    if len(incomes) == 1:
        return incomes[0]
    return 8


#### Find BANDA subjects

In [ ]:
subs = df_all.subject_id.dropna().apply(lambda x: np.nan if "ineligible" in x 
                                 or'-' in x 
                                 or int(x) > 900 
                                 or int(x) < 100
                                 else int(x)).astype(pd.Int64Dtype()).dropna()

In [ ]:
banda_subs = df_ids[df_ids.subject_id.isin(subs)].banda_id.dropna().str.contains('BANDA')
banda_subs_idx = banda_subs[banda_subs].index

banda_df_ids = df_ids.loc[banda_subs_idx]
sub_ids = banda_df_ids.subject_id

In [ ]:
df_subs = subs.isin(banda_df_ids.subject_id)
df_subs_idx = df_subs[df_subs].index

#### Create a BANDA specific dataframe

In [ ]:
df_banda = df_all.loc[df_subs_idx]
df_banda["src_subject_id"] = df_banda.subject_id.dropna().apply(lambda x: banda_df_ids[banda_df_ids.subject_id == int(x)].banda_id.values[0])

# combine tanner girls and boys for processing
tanner_cols = [var.replace("_boys", "").replace("_girls", "") for var in df_banda.columns if re.match("tanner\d+_(boys|girls)", var)]
for val in list(sorted(set(tanner_cols))):
    df_banda[val] = df_banda[val.replace("_", "_boys_")].add(df_banda[val.replace("_", "_girls_")], fill_value=0)
df_banda

In [ ]:
def to_timestamp(x):
    """Convert encoded timestamps to a standard form"""
    if '[' in x:
        return np.nan
    #if ":" in x or '-' in x:
    return dateutil.parser.parse(x)
    #return np.nan

In [ ]:
def assess2df(df, assess, ts_regex, recode=True, skip_suffix=False):
    """Convert each assessment to a dataframe
    
    This function also recodes values by applying the relevant transforms.
   
    """
    _, _, suffixmap, suffixmap_ts = get_timemap(df, ts_regex)
    df_assess = []
    for tp, tcol in suffixmap_ts.items():
        tpext = tp
        if skip_suffix:
            tpext = ""
        if isinstance(assess, str):
            df_assess.append(pd.concat([df[["src_subject_id", "subject_id", tcol]], 
                                        df.filter(regex=f"{assess}{tpext}$")], axis=1))
        elif isinstance(assess, list):
            df_assess.append(pd.concat([df[["src_subject_id", "subject_id", tcol]], 
                                        df.filter(items=[f"{var}{tpext}" for var in assess])], axis=1))
        else:
            raise ValueError("assess parameter must be a regex or a list")
        df_assess[-1]["visit"] = suffixmap[tp]
        df_assess[-1]["interview_date"] = df_assess[-1][tcol].dropna().apply(lambda x: to_timestamp(x))
        df_assess[-1] = df_assess[-1].drop(tcol, axis=1)
        if not skip_suffix:
            columns = [x.removesuffix(tp) for x in df_assess[-1].columns]
            df_assess[-1].columns = columns
        df_assess[-1].sort_values("src_subject_id", inplace=True)
        df_assess[-1] = df_assess[-1].reset_index(drop=True)
    if len(df_assess) > 1:
        df_all = pd.concat(df_assess, axis=0, ignore_index=True)
    else:
        df_all = df_assess[0]
    if recode:
        for var in df_all:
            if var in var_map:
                varinfo = df_maps[var_map[var]][df_maps[var_map[var]].banda_var == var]
                if len(varinfo.recode.dropna()):
                    recode_func = globals()[varinfo.recode.values[0]]
                    df_all[var] = df_all[var].dropna().apply(lambda x: recode_func(x))
                if len(varinfo.na_val.dropna()):
                    if varinfo.na_val.values[0] in df_all[var].tolist():
                        df_all[var].loc[df_all[var] == varinfo.na_val.values[0], var] = np.nan
                if varinfo.ndar_var.values[0] != var:
                    df_all = df_all.rename(columns={var: varinfo.ndar_var.values[0]})
    return df_all

In [ ]:
def drop_allnans(df, cols):
    """Drop data connecting NaNs"""
    allnans = df[cols].isnull().all(axis=1)
    somedata = allnans[allnans == False]
    return df.loc[somedata.index]

#### Process the adolescent assessments

In [ ]:
ts_regex = "adolescent.*"
print(get_timemap(df_banda, ts_regex))
dfs_assess = {}
for assess in ["bisbas",
               "handed",
               "mfq", 
               "neo", 
               "rbqa", 
               "rcads",
               "shaps", 
               "stai_(state|trait)", 
               "tanner"]:
    assess_strip = assess.split("\\")[0].split(".")[0].split("_")[0]
    dfs_assess[assess_strip] = assess2df(df_banda, f"{assess}\d+", ts_regex)
    cols = np.setdiff1d(dfs_assess[assess_strip].columns, nda_vars)
    dfs_assess[assess_strip] = drop_allnans(dfs_assess[assess_strip], cols)
    dfs_assess[assess_strip]["respondent"] = "Child"

#### Process the parent assessments

In [ ]:
parent_subs = df_parents.subject_id.dropna().apply(lambda x: np.nan if "ineligible" in x 
                                                   or '-' in x 
                                                   or 'HP' in x
                                                   or int(x) > 900 
                                                   or int(x) < 100
                                                   else int(x)).astype(pd.Int64Dtype()).dropna()

In [ ]:
df_parent_subs = parent_subs.isin(sub_ids)
df_parent_subs_idx = df_parent_subs[df_parent_subs].index
df_parent = df_parents.loc[df_parent_subs_idx]
df_parent.subject_id = df_parent.subject_id.apply(lambda x: int(x))
df_parent["src_subject_id"] = df_parent.subject_id.dropna().apply(lambda x: banda_df_ids[banda_df_ids.subject_id == int(x)].banda_id.values[0])

# collapse categorical variables
catvars = np.unique([var.split("__")[0] for var in df_parent.columns if "__" in var]).tolist()
for catvar in catvars:
    varcols = sorted([var for var in df_parent.columns if var.startswith(catvar + "__")])
    df_parent[catvar] = df_parent[varcols].values.tolist() #.dot([2** var for var in range(len(varcols))])

df_parent.drop(columns=['demo_child_gender_i', 'demo_other_source_i', 'demo_income_i', 'demo_child_i'],
               inplace=True)
df_parent

In [ ]:
ts_regex_par = "parent.*"
print(get_timemap(df_parent, ts_regex_par))

dfs_assess_par = {}
for assess in ["cbcl.*pr\d+([a-z]*)", #child
               "demo.*(?<!__\d)$", 
               "masq\d+", #self
               "mfq.*pr\d+", #child
               "rmbi.*pr\d+", #child
               "stai.*pr\d+"]: #self
    assess_strip = assess.split("\\")[0].split(".")[0]
    if assess_strip == "demo":
        dfs_assess_par[assess_strip] = assess2df(df_parent, assess, ts_regex_par, skip_suffix=True)
    else:
        dfs_assess_par[assess_strip] = assess2df(df_parent, assess, ts_regex_par)
    cols = np.setdiff1d(dfs_assess_par[assess_strip].columns, nda_vars)
    dfs_assess_par[assess_strip] = drop_allnans(dfs_assess_par[assess_strip], cols)
    dfs_assess_par[assess_strip]["respondent"] = "Parent"

In [ ]:
dfs_assess_par["demo"]

#### Process all the clinical assessments

In [ ]:
clinic_subs = df_clinics.subject_id.dropna().apply(lambda x: np.nan if "ineligible" in x.lower() 
                                                   or '-' in x 
                                                   or 'HP' in x
                                                   or 'illegible' in x
                                                   or int(x) > 900 
                                                   or int(x) < 100
                                                   else int(x)).astype(pd.Int64Dtype()).dropna()

In [ ]:
df_clinic_subs = clinic_subs.isin(sub_ids)
df_clinic_subs_idx = df_clinic_subs[df_clinic_subs].index
df_clinic = df_clinics.loc[df_clinic_subs_idx]
df_clinic.subject_id = df_clinic.subject_id.apply(lambda x: int(x))
df_clinic["src_subject_id"] = df_clinic.subject_id.dropna().apply(lambda x: banda_df_ids[banda_df_ids.subject_id == int(x)].banda_id.values[0])
df_clinic

In [ ]:
ksads1_var = df_maps[-1][(df_maps[-1].assessment == 'KSADS') & (df_maps[-1].ndar_structure == 'ksads_diagnoses01')].banda_var.dropna().unique().tolist()
ksads2_var = df_maps[-1][(df_maps[-1].assessment == 'KSADS') & (df_maps[-1].ndar_structure == 'ksads_diagnosesp201')].banda_var.dropna().unique().tolist()

df_other = {}
df_ksads1 = assess2df(df_clinic, ksads1_var, "ksads.*")
cols = np.setdiff1d(df_ksads1.columns, nda_vars)
df_ksads1 = drop_allnans(df_ksads1, cols)
dsm_drop_vars = [var for var in df_ksads1.columns if (var + "_dsm5") in df_ksads1.columns] + \
        ["bipolarunspecpast_dsm5", "briefreactivepsychosiscurrent", "depressunspecpast_dsm5", 
         "schziophreniacurrent", "schziophreniformcurrent"]
df_ksads1 = df_ksads1.drop(dsm_drop_vars, axis=1)
df_other["ksads_diagnoses01"] = df_ksads1

df_ksads2 = assess2df(df_clinic, ksads2_var, "ksads.*")
cols = np.setdiff1d(df_ksads2.columns, nda_vars)
df_ksads2 = drop_allnans(df_ksads2, cols)
df_other["ksads_diagnosesp201"] = df_ksads2
    
df_fhs = assess2df(df_clinic, "fhs\d+.*", "fhs.*")
cols = np.setdiff1d(df_fhs.columns, nda_vars)
df_fhs = drop_allnans(df_fhs, cols)
df_other["fhs"] = df_fhs

df_cssrs = assess2df(df_clinic, "cssrs_.*", "cssrs.*")
describe_vars = [var for var in df_cssrs.columns if "describe" in var]
df_cssrs = df_cssrs.drop(describe_vars, axis=1)
cols = np.setdiff1d(df_cssrs.columns, nda_vars)
df_cssrs = drop_allnans(df_cssrs, cols)
unk_cols = [var for var in df_cssrs.columns if "_sincelastvisit" in var] + \
        ["cssrs_ideation_severe_intensity_recent", "cssrs_ideation_severe_desc_recent",
         "cssrs_ideation_duration_recent", "cssrs_ideation_control_recent", 
         "cssrs_ideation_deter_recent", "cssrs_ideation_reason_recent",
         "cssrs_actual_leth_lethal_date_recent", "cssrs_actual_leth_lethal_code_recent",
         "cssrs_potential_leth_lethal_code_recent", "cssrs_case_note"]
df_cssrs = df_cssrs.drop(unk_cols, axis=1)
df_other["cssrs"] = df_cssrs

wasi_var = df_maps[-1][(df_maps[-1].assessment == 'WASI') & (df_maps[-1].ndar_structure == 'wasi201')].banda_var.dropna().unique().tolist()
wasi_c_var = [var for var in wasi_var if not var.endswith("_p")]
wasi_p_var = [var for var in wasi_var if var.endswith("_p")]
wasi_c_var, wasi_p_var

df_wasi_c = assess2df(df_clinic, wasi_c_var, "wasi_init.*", skip_suffix=True)
cols = np.setdiff1d(df_wasi_c.columns, nda_vars)
df_wasi_c = drop_allnans(df_wasi_c, cols)
df_other["wasi_c"] = df_wasi_c

df_wasi_p = assess2df(df_clinic, wasi_p_var, "wasi_parent.*", skip_suffix=True)
cols = np.setdiff1d(df_wasi_p.columns, nda_vars)
df_wasi_p = drop_allnans(df_wasi_p, cols)
df_other["wasi_p"] = df_wasi_p

for key in df_other:
    df_other[key]["respondent"] = "Child" if "_p" not in key else "Parent"

#### Create a visit structure for all timepoints and subjects

In [ ]:
visit_dates = []
for val in dfs_assess.values():
    visitinfo = val[["src_subject_id", "visit", "interview_date"]].drop_duplicates()
    visitinfo = visitinfo.set_index(keys=["src_subject_id", "visit"])
    visit_dates.append(visitinfo)
df_visits = pd.concat(visit_dates, axis=1)

visit_dates_par = []
for val in dfs_assess_par.values():
    visitinfo = val[["src_subject_id", "visit", "interview_date"]].drop_duplicates()
    visitinfo = visitinfo.set_index(keys=["src_subject_id", "visit"])
    visit_dates_par.append(visitinfo)
df_visits_par = pd.concat(visit_dates_par, axis=1)

df_visits = pd.concat(visit_dates + visit_dates_par, axis=1)

In [ ]:
df_visit = df_visits.apply(lambda x: pd.to_datetime(sorted(x.dropna().unique())[0]) if len(x.dropna().unique()) >= 1 else pd.NaT, axis=1).to_frame()
df_visit.columns = ["interview_date"]


all_date_missing = df_visit[df_visit.interview_date.isnull()]
T1_missing_loc = all_date_missing.index.get_locs([slice(None), "T1"])
T1_missing_subs = all_date_missing.iloc[T1_missing_loc].index.get_level_values("src_subject_id").tolist()
T2_missing_loc = all_date_missing.index.get_locs([slice(None), "T2"])
T2_missing_subs = all_date_missing.iloc[T2_missing_loc].index.get_level_values("src_subject_id").tolist()
T3_missing_loc = all_date_missing.index.get_locs([slice(None), "T3"])
T3_missing_subs = all_date_missing.iloc[T3_missing_loc].index.get_level_values("src_subject_id").tolist()

for val in T1_missing_subs:
    timestamp = pd.to_datetime(old_subj_info[old_subj_info.src_subject_id == val].interview_date)
    df_visit.loc[pd.IndexSlice[(val, "T1")], "interview_date"] = timestamp.values[0]
for val in T2_missing_subs:
    timestamp = df_visit.loc[pd.IndexSlice[(val, "T1")], "interview_date"]
    df_visit.loc[pd.IndexSlice[(val, "T2")], "interview_date"] = timestamp + relativedelta(months=6)
for val in T3_missing_subs:
    timestamp = df_visit.loc[pd.IndexSlice[(val, "T1")], "interview_date"]
    df_visit.loc[pd.IndexSlice[(val, "T3")], "interview_date"] = timestamp + relativedelta(months=12)    
df_visit[df_visit.interview_date.isnull()]

In [ ]:
def diff_month(d1, d2):
    return int((d1.year - d2.year) * 12 + d1.month - d2.month)

def get_age(x):
    dob = pd.to_datetime(df_ids[df_ids.banda_id == x.name[0]].dob.values[0])
    return diff_month(x.interview_date, dob)

def get_gender(x):
    val = banda_df_ids[banda_df_ids.banda_id == x.name[0]].sex.values
    # demo = dfs_assess_par["demo"] 
    # demo = demo[demo.visit == "T1"]
    # val = demo[demo.src_subject_id == x.name[0]].demo_child_gender_i.values
    if len(val):
        val = val[0]
        return "F" if val < 0.5 else "M"
    return np.nan
    
df_visit["interview_age"] = df_visit.dropna(subset=["interview_date"]).apply(get_age, axis=1)
df_visit["gender"] = df_visit.apply(get_gender, axis=1)
df_visit

#### Create a flat visit mapping table that includes GUIDS

In [ ]:
subj_flatten = pd.concat((df_visit.index.to_frame(), df_visit), axis=1)
subj_flatten["subjectkey"] = subj_flatten.src_subject_id.map(subjmap)
subj_flatten = subj_flatten[["subjectkey", "src_subject_id", "interview_date", "interview_age", "gender", "visit"]]

def get_numeric_id(x):
    subjects = df_banda[df_banda.src_subject_id==x][["src_subject_id", "subject_id"]].drop_duplicates().subject_id.unique().tolist()
    if len(subjects) > 1:
        return np.nan
    return subjects.pop()
subj_flatten["numeric_id"] = subj_flatten.src_subject_id.apply(get_numeric_id)
subj_flatten.interview_date = subj_flatten.interview_date.apply(lambda x: datetime.strftime(x, "%m/%d/%Y"))
                                  
subj_flatten

In [ ]:
def combine_subj_info(df):
    """Add NDAR variables to response tables"""
    foo = df.set_index(keys=["src_subject_id", "visit"])
    for key in ["subject_id", "interview_date"]:
        if key in foo.columns:
            foo = foo.drop(columns=[key])
    if "respondent" in foo.columns:
        foo = foo[["respondent"] + [var for var in foo.columns.tolist() if var != "respondent"]]
    return pd.concat((subj_flatten.drop(columns=["numeric_id"]), foo), axis=1, join="inner")

In [ ]:
# Generate a unique output dir for each run of this script
outdir = (Path() / f"output-{datetime.now()}")
outdir.mkdir()
outdir

In [ ]:
# Remap Redcap names to NDA structure names
remapkey = {"handed": "chaphand", 
            "neo": "nffi",
            "ksads_diagnoses01": "diagnoses01",
            "ksads_diagnosesp201": "diagnosesp201"}

def insert_dictname(outfile, dictname, version):
    """Insert the structure name into the CSV to comply with NDA structures"""
    with open(outfile) as fp:
        table = fp.readlines()
    extra_commas = len([1 for v in table[0] if v == ","]) - 1
    table.insert(0, f"{dictname},{version}{',' * extra_commas}\n")
    with open(outfile, "w") as fp:
        fp.writelines(table)


def write_csv(dfs, outdir, structnamemap={}):
    """Write a dataframe into the corresponding NDA structure csv"""
    for key, df in dfs.items():
        dfkey = combine_subj_info(df)
        structname = structnamemap.get(key)
        if structname is None:
            if key in remapkey:
                key = remapkey[key]
            for val in assess_map:
                if key.split("_")[0] in val:
                    structname = val
                    continue
            if structname is None:
                raise ValueError(f"Could not find {key} in assess_map")
        print(key, key.split("_")[0], structname)
        outfile = outdir / f"{structname}.csv"
        if outfile.exists():
            df = pd.read_csv(outfile, skiprows=[0])
            dfkey = pd.concat((df, dfkey), axis=0, ignore_index=True)
        dfkey.reset_index(drop=True, inplace=True)
        if "respondent" in dfkey:
            dfkey = dfkey.sort_values(["respondent", "visit", "src_subject_id"],
                                      ignore_index=True)
        else:
            dfkey = dfkey.sort_values(["visit", "src_subject_id"],
                                      ignore_index=True)
        dfkey = dfkey.convert_dtypes()
        if "ksads" in str(outfile):
            dfkey.to_csv(outfile, index=False, na_rep=0)
        else:
            dfkey.to_csv(outfile, index=False, na_rep=999)
        dictname, version = structname[:-2],structname[-2:]
        print(dictname, version)
        insert_dictname(outfile, dictname, int(version))
        structname = None

#### Write data into existing or new files

In [ ]:
write_csv(dfs_assess, outdir)
write_csv(dfs_assess_par, outdir)
write_csv(df_other, outdir)

### Process other data sources

1. NIH toolbox
1. STRAIN 
1. Penn cognitive battery

#### NIH toolbox

Multiple CSVs refer to the different sources that were accumulated between
the receiving server and exported directly from the IPads

In [ ]:
nihcsvs = glob("../../../../../NIHtoolbox/*Scores*csv_*") + \
        glob("../../../../../remaining_nihtoolbox_data/*/*Scores.csv")
len(nihcsvs)

In [ ]:
nih_dfs = []
for filename in nihcsvs:
    nih_dfs.append(pd.read_csv(filename))
df_nih = pd.concat(nih_dfs, axis=0, ignore_index=True).drop_duplicates()
df_nih.shape

In [ ]:
def to_subject_id(x):
    if x in ["Gyz", "Train123", "Test", "Testt", "Viv", "Test6", "Test 4", "Test3"]:
        return np.nan
    x = int(x)
    if x < 100 or x > 900 or not x in banda_df_ids.subject_id.dropna().values.astype(int):
        return np.nan
    return x
df_nih['subject_id'] = df_nih.PIN.dropna().apply(to_subject_id)
df_nih["src_subject_id"] = df_nih.subject_id.dropna().apply(lambda x: banda_df_ids[banda_df_ids.subject_id == int(x)].banda_id.values[0])
df_nih = df_nih.dropna(subset=["src_subject_id"])

In [ ]:
df_nih[['PIN', "src_subject_id", 'Inst', 'RawScore', 'ItmCnt', 'DateFinished', 
        'Uncorrected Standard Score', 'Age-Corrected Standard Score',
        'Fully-Corrected T-score', 'National Percentile (age adjusted)']].head()

In [ ]:
df_nih.Inst.unique()

In [ ]:
nihtasks = ["List", "Flanker", "Dimensional", "Oral", "Pattern"]
columns = ['RawScore', 'ItmCnt', 'Uncorrected Standard Score', 'Age-Corrected Standard Score',
        'Fully-Corrected T-score']

taskmap = {}
for val in df_nih.Inst.unique():
    if "Flanker" in val:
        task = splitname = "Flanker"
    else:
        splitname = ".".join(val.split("Toolbox ")[1].split("Test")[0].split() + ["Test"])
        task = splitname.split(".")[0]
    taskmap[task] = splitname

def name2var(task, col):
    if col == "RawScore":
        col = "Raw Score"
    cols = col.replace("T-s", "T S").replace("-", " ").split()
    return ".".join([taskmap[task]] + cols)

def get_tbx_df(df_nih, task, colmap):
    df_nih_task = df_nih[df_nih.Inst.dropna().apply(lambda x: task in x)]
    df_nih_task = df_nih_task[colmap.keys()]
    df_nih_task.columns = colmap.values()
    df_nih_task["visit"] = "T1"
    df_nih_task = df_nih_task.drop_duplicates()
    return df_nih_task.drop_duplicates(subset=["src_subject_id", "visit"])

df_nihtbx = {}
for task in nihtasks:
    colmap = {"Inst": "version_form", "src_subject_id": "src_subject_id"}
    for col in columns:
        banda_var = name2var(task, col)
        info_idx = var_map[banda_var]
        df2use = df_maps[info_idx]
        row = df2use[df2use["banda_var"] == banda_var]
        ndar_var = row.ndar_var
        colmap[col] = ndar_var.values[0]
    key = row.ndar_structure.values[0]
    df_nihtbx[key] = get_tbx_df(df_nih, task, colmap)

In [ ]:
for grp, subdf in df_nihtbx['lswmt01'].groupby("src_subject_id"):
    if subdf.shape[0] > 1:
        print(grp, subdf)

In [ ]:
write_csv(df_nihtbx, outdir)

#### Convert STRAIN data into NDA

In [ ]:
df_strain = pd.read_csv("STRAIN-BANDA_09-06-22.csv")
df_strain.head()

In [ ]:
def to_subject_id(x):
    if "test" in x:
        return np.nan
    x = int(x)
    if x < 100 or x > 900 or not x in banda_df_ids.subject_id.dropna().values.astype(int):
        return np.nan
    return x
columns = df_strain.columns[5:]
df_strain['subject_id'] = df_strain.ID.dropna().apply(to_subject_id)
df_strain["src_subject_id"] = df_strain.subject_id.dropna().apply(lambda x: banda_df_ids[banda_df_ids.subject_id == int(x)].banda_id.values[0])
df_strain = df_strain.dropna(subset=["src_subject_id"])
colmap = {"src_subject_id": "src_subject_id"}
varnomap = []
for banda_var in columns:
    if banda_var not in var_map:
        varnomap.append(banda_var)
        continue
    info_idx = var_map[banda_var]
    df2use = df_maps[info_idx]
    row = df2use[df2use["banda_var"] == banda_var]
    ndar_var = row.ndar_var
    colmap[banda_var] = ndar_var.values[0]
print("Missing in NDAR: ", varnomap)
df_strain = df_strain[colmap.keys()]
df_strain.columns = colmap.values()
df_strain["respondent"] = "Child"
df_strain["visit"] = "T1"



In [ ]:
df_strain

In [ ]:
df_strain.head()

In [ ]:
write_csv({"strain01": df_strain}, outdir)

#### Convert Penn Toolbox data into NDA

In [ ]:
df_penn = pd.read_csv("penn_all_banda214.csv")
for key in ["PMAT24_A.PMAT24_A_RTCR", "PMAT24_A.PMAT24_A_RTER", "PMAT24_A.PMAT24_A_RTTO"]:
        df_penn[key] = df_penn[key].dropna().apply(lambda x: np.round(x))
df_penn.head()

In [ ]:
penntasks = ["Penn Word Memory Test", "Penn Matrix Reasoning Test", "Penn Emotion Regonition Test",
            "Delay Discounting Task"]
pennvars = ["KCPW_A.", "PMAT24_A.", "ER40_D.", "DDISC."]
intvars = ["er40_c_cr", "er40_d_fc", "er40_d_mc", "er40_c_ang",
           "er40_c_fear", "er40_c_hap", "er40_c_noe", "er40_c_sad"]

def get_task_df(df_p, colmap):
    df_penn_task = df_p[colmap.keys()].copy()
    df_penn_task.columns = colmap.values()
    df_penn_task["respondent"] = "Child"
    df_penn_task["visit"] = "T1"
    df_penn_task = df_penn_task.drop_duplicates()
    return df_penn_task.drop_duplicates(subset=["src_subject_id", "visit"])

df_penntbx = {}
for task in pennvars:
    colmap = {"banda_id": "src_subject_id"}
    taskcols = [var for var in df_penn.columns if var.startswith(task)]
    for banda_var in taskcols:
        if banda_var not in var_map:
            continue
        info_idx = var_map[banda_var]
        df2use = df_maps[info_idx]
        row = df2use[df2use["banda_var"] == banda_var]
        ndar_var = row.ndar_var
        colmap[banda_var] = ndar_var.values[0]
    key = row.ndar_structure.values[0]
    df_penntbx[key] = get_task_df(df_penn, colmap)

In [ ]:
df_penntbx[key].head(11)

In [ ]:
write_csv(df_penntbx, outdir)

### Write subject structure (ndar_subject01) with diagnostic groups

**Anxious:** current diagnosis of at least one of {anx} disorders and no current dx of any {dep} disorders

- IF diagnoses_current_1=1; AND dx_current_2__{anx}=1; AND NOT dx_current_2__{dep}=1
- Where {anx} = agorophobia; gen_anxiety; panic; sep_anxiety; social_phobia; specific_phobia
- No lifetime consideration

**Depressed:** current dx of at least one {dep} disorders

- IF diagnoses_current_1=1; AND dx_current_2__{dep}=1
- Where {dep} = adj_depressed; depressive_nos; dysthymia; mdd
- No {anx} current nor any lifetime consideration

**Control:** No current disorder, no lifetime {anx} nor {dep} disorder

- IF diagnoses_current_1=0; AND diagnoses_lifetime_1=0; OR 
- IF diagnoses_current_1=0; AND diagnoses_lifetime_1=1 NOT {anx} nor {dep} lifetime

In [ ]:
race_map = {1: "American Indian/Alaskan Native",
            2: "Asian",
            3: "Black or African American", 
            4: "Hawaiian or Pacific Islander",
            5: "White",
            6: "More than one race",
            7: "Unknown or not reported"}
ethnic_map = {1: "Hispanic or Latino",
              2: "Not Hispanic or Latino",
              3: "Unknown or not reported"}

In [ ]:
anx = [f"dx_current_2___{anx}" for anx in ["agoraphobia", "gen_anxiety", "panic", "sep_anxiety", "social_phobia", "specific_phobia", "other_spec_anxiety"]]
dep = [f"dx_current_2___{dep}" for dep in ["adj_depressed", "depressive_nos", "dysthymia", "mdd"]]

In [ ]:
shortname = "ndar_subject01"

fields = ["phenotype", "phenotype_description"]

df_subject = banda_df_ids[["banda_id", "race", "ethnicity"]].copy()
df_subject.race = df_subject.race.apply(lambda x: race_map[x])
df_subject.ethnicity = df_subject.ethnicity.apply(lambda x: ethnic_map[x])
df_subject.columns = ["src_subject_id", "race", "ethnic_group"]
df_subject["phenotype"] = "Unknown"
df_subject["phenotype_description"] = """dx= partial remission (3) or definite (4)
{anx}= agorophobia; gen_anxiety; panic; sep_anxiety; social_phobia; specific_phobia;
other_spec_anxiety
{dep}= adj_depressed; depressive_nos; dysthymia; mdd
**Anxiety:** at least one current {anx} dx and NO current {dep} dx
- IF diagnoses_current_1=1; AND dx_current_2__{anx}=1; AND NOT dx_current_2__{dep}=1
- No lifetime dx consideration for label
**Depression:** at least one current {dep} dx, MUST be present at time of interview to meet
label criteria (eg, met dx threshold for major depressive episode at intake)
- IF diagnoses_current_1=1; AND dx_current_2__{dep}=1
- {anx} dx is NOT exclusionary for label; No lifetime dx consideration for label
**Control:** No current dx, no current or lifetime {anx} nor {dep} dx
- IF diagnoses_current_1=0; AND diagnoses_lifetime_1=0; OR
- IF diagnoses_current_1=0; AND diagnoses_lifetime_1=1 NOT {anx} nor {dep} dx lifetime"""
anx_subs = ((df_ids["diagnoses_current_1"] == 1) & (df_ids[anx].sum(axis=1) > 0) & (df_ids[dep].sum(axis=1) == 0))
anx_idx = df_ids[anx_subs].banda_id
dep_subs = ((df_ids["diagnoses_current_1"] == 1) & (df_ids[dep].sum(axis=1) > 0))
dep_idx = df_ids[dep_subs].banda_id
ctrl_subs = ((df_ids["diagnoses_current_1"] == 0) & (df_ids["dx_lifetime_1"] == 0)) | \
            ((df_ids["diagnoses_current_1"] == 0) & (df_ids["dx_lifetime_1"] == 1) \
             & (~ ((df_ids[dep].sum(axis=1) > 0) | (df_ids[anx].sum(axis=1) > 0))))
ctrl_idx = df_ids[ctrl_subs].banda_id
df_subject.loc[df_subject.src_subject_id.isin(anx_idx), "phenotype"] = "Anxiety"
df_subject.loc[df_subject.src_subject_id.isin(dep_idx), "phenotype"] = "Depression"
df_subject.loc[df_subject.src_subject_id.isin(ctrl_idx), "phenotype"] = "Control"
df_subject["visit"] = "T1"
df_subject["twins_study"] = "No"
df_subject["sibling_study"] = "No"
df_subject["family_study"] = "No"
df_subject["sample_taken"] = "No"
df_subject.sort_values("src_subject_id", inplace=True)
df_subject

In [ ]:
df_subject.phenotype.hist();

In [ ]:
df_unk = df_ids.loc[df_ids.banda_id.isin(df_subject[df_subject.phenotype == "Unknown"].src_subject_id), ["banda_id", "diagnoses_current_1"] + anx + dep]
df_unk

In [ ]:
write_csv({shortname: df_subject}, outdir, {shortname: shortname})

#### Count all the outputs produced

In [ ]:
outcsvs = sorted(glob(str(outdir / "*.csv")))
len(outcsvs)

Download the required data elements for each NDA structure
and rewrite the CSVs with missing data filled in.

In [ ]:
if Path("required_de.json").exists():    
    with open("required_de.json") as fp:
        required_des = json.load(fp)
else:
    ndar_dicts = [Path(val).name.split(".csv")[0] for val in outcsvs]
    required_des = {}
    for shortname in ndar_dicts:
        info = requests.get(f"https://nda.nih.gov/api/datadictionary/datastructure/{shortname}").json()
        required_des[shortname] = {}
        for de in info["dataElements"]:
            if de["required"] == "Required":
                required_des[shortname][de["name"]] = de["aliases"]
    with open("required_de.json", "wt") as fp:
        json.dump(required_des, fp, indent=2)

In [ ]:
def add_missing(filename, shortname, elements):
    df_name = pd.read_csv(filename, na_values=[999], skiprows=[0])
    for el in elements:
        df_name[el] = np.nan
    df_name = df_name.convert_dtypes()
    df_name.to_csv(filename, index=False, na_rep=999)
    insert_dictname(filename, shortname[:-2], int(shortname[-2:]))

In [ ]:
for name, req_els in required_des.items():
    csvname = outdir / f"{name}.csv"
    df_name = pd.read_csv(csvname, na_values=[999], skiprows=[0])
    missing_de = []
    for val, aliases in req_els.items():
        if val in df_name.columns:
            continue
        if any([alias in df_name.columns for alias in aliases]):
            continue
        missing_de.append(val)
    print(name, missing_de)
    if missing_de:
        add_missing(csvname, name, missing_de)

### Process composite scores

1. BISBAS
1. MFQ
1. NEO FFI
1. RBQA
1. RCADS
1. SHAPS
1. STAI
1. CBCL
1. MASQ
1. RMBI

Each section contains code to remap NDA data elements to the scoring needed
to generate the composite scores

#### Bisbas

In [ ]:
csvname = outdir / "bisbas01.csv"
bisbas_dict = dict(bissc_total=[2,8,13,16,19,22,24],
                   bas_drive=[3, 9, 12, 21],
                   bas_fs=[5, 10, 15, 20],
                   bas_rr=[4, 7, 14, 18, 23])
bisbas_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
for key, val in bisbas_dict.items():
    columns = [f"bisbas{v}" for v in val]
    bisbas_df[key] = bisbas_df[columns].sum(axis=1)
bisbas_df["version_form"] = "2013"
bisbas_df = bisbas_df.convert_dtypes()
bisbas_df.to_csv(csvname, index=False)
display(bisbas_df.head())
insert_dictname(csvname, "bisbas", 1)

#### MFQ (LONG VERSION)

For adolescent self-report:

`mfqtot = sum(items[1-33])`

For parent report on adolescent:

`mfqtot = sum(items[1-34])`

NDA codes this questionnaire differently from the standard questionnaire. 

- MFQ score scale: Not True = 0, Sometimes = 1, True =2
- NDA score scale: Not True = 1, Sometimes = 2, True =3

Hence for NDA total score a 1 has to be subtracted from each entry before computing total score. See Daviss et al. for details. https://acamh.onlinelibrary.wiley.com/doi/pdf/10.1111/j.1469-7610.2006.01646.x?casa_token=dMiq1Fx3CqAAAAAA:haUEBr0WtiMdBzrboWFZrEEAO6cJzojckjIW8qLD0_nB94ckAXiuLvtfdZ2nwj60hLu3PhxhVEPLGg  


In [ ]:
csvname = outdir / "mfq01.csv"
mfq_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
columns = [f"mfq{v}" for v in range(1, 35)]
mfq_df["mfqtot"] = (mfq_df[columns] - 1).sum(axis=1)
mfq_df = mfq_df.convert_dtypes()
mfq_df.to_csv(csvname, index=False, na_rep=999)
display(mfq_df.head())
insert_dictname(csvname, "mfq", 1)
mfq_df[["respondent", "mfqtot"]].groupby("respondent").hist();

#### NEO FFI

reverse score: [1, 4, 7, 10] then sum remaining items of the 12 neo items.

NDA Formula: 

```
neo2_score_ne: reverse(nffi_1) + reverse(nffi_16) + reverse(nffi_31) + reverse(nffi_46) + nffi_6 + nffi_11 + 
nffi_21 + nffi_26 + nffi_36 + nffi_41 + nffi_51 + nffi_56
```

In [ ]:
csvname = outdir / "nffi01.csv"
nffi_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
reverse_nffi = lambda x: 6 - x
nffi_df["neo2_score_ne"] = nffi_df[["neo_ne1", "neo_ne4", "neo_ne7", "neo_ne10"]].apply(reverse_nffi).sum(axis=1) + \
     nffi_df[["neo_ne2", "neo_ne3", "neo_ne5", "neo_ne6", "neo_ne8", "neo_ne9", "neo_ne11", "neo_ne12"]].sum(axis=1)
nffi_df = nffi_df.convert_dtypes()
nffi_df.to_csv(csvname, index=False, na_rep=999)
display(nffi_df.head())
insert_dictname(csvname, "nffi", 1)

#### RBQA

```
rbqa_total (total score): sum for rbqa<items> for total 
rbqa_scale1 (unsafe sexual practices): 2, 12 
rbqa_scale2 (aggressive and/or violent behaviors): 3, 4, 13 
rbqa_scale3 (rule breaking): 8, 9, 17 
rbqa_scale4 (dangerous, destructive, and/or illegal): 1, 10, 11, 16, 18, 20 
rbqa_scale5 (self-injurious behaviors): 14, 15 
rbqa_scale6 (alcohol and/or drug use): 5, 6, 7, 19 
```

See Auerbach and Gardiner, 2012: https://cdasr.mclean.harvard.edu/wp-content/uploads/2017/09/Auerbach_2012_BRT.pdf 

In [ ]:
csvname = outdir / "rbqa01.csv"
rbqa_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
rbqa_dict = dict(rbqa_total=list(range(1, 21)),
                 rbqa_scale1=[2, 12],
                 rbqa_scale2=[3, 4, 13],
                 rbqa_scale3=[8, 9, 17],
                 rbqa_scale4=[1, 10, 11, 16, 18, 20],
                 rbqa_scale5=[14, 15],
                 rbqa_scale6=[5, 6, 7, 19]
                )
for key, val in rbqa_dict.items():
    columns = [f"rbqa{v}" for v in val]
    rbqa_df[key] = rbqa_df[columns].sum(axis=1)
rbqa_df = rbqa_df.convert_dtypes()
rbqa_df.to_csv(csvname, index=False, na_rep=999)
display(rbqa_df.head())
insert_dictname(csvname, "rbqa", 1)

#### RCADS

from Chorpita et al., (2015):
```
rcads_social_phob: 4, 7, 8, 12, 20, 30, 32, 38, 43 
rcads_panic: 3, 14, 24, 26, 28, 34, 36, 39, 41 
rcads_drs (major depression): 2, 6, 11, 15, 19, 21, 25, 29, 40, 47 
rcads_sep_anx: 5, 9, 17, 18, 33, 45, 46 
rcads_gen_anx: 1, 13, 22, 27, 35, 37 
rcads_ocd: 10, 16, 23, 31, 42, 44 
```

Even though RCADS uses strings in the data dictionary it accepts ordinal values associated with it. 

In NDA definitions, but not computed here: `rcads_ars, rcads_total_int`

In [ ]:
csvname = outdir / "rcads01.csv"
rcads_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
rcads_dict = dict(rcads_tot=list(range(1, 48)),
                  rcads_social_phob=[4, 7, 8, 12, 20, 30, 32, 38, 43],
                  rcads_panic=[3, 14, 24, 26, 28, 34, 36, 39, 41],
                  rcads_drs=[2, 6, 11, 15, 19, 21, 25, 29, 40, 47],
                  rcads_sep_anx=[5, 9, 17, 18, 33, 45, 46],
                  rcads_gen_anx=[1, 13, 22, 27, 35, 37],
                  rcads_ocd=[10, 16, 23, 31, 42, 44],
                )
for key, val in rcads_dict.items():
    columns = [f"rcads_{v}" for v in val]
    rcads_df[key] = rcads_df[columns].sum(axis=1)
rcads_df = rcads_df.convert_dtypes()
rcads_df.to_csv(csvname, index=False, na_rep=999)
display(rcads_df.head())
insert_dictname(csvname, "rcads", 1)

#### SHAPS

See Snaith et al., 1995 for original "dichotomous scoring"; Franken et al., 2007 for "continuous scoring".

- Orig scale: 0 Strongly Disagree – 3 Strongly Agree 
- Scale for items that needs to be reversed: 0 Definitely Agree – 3 Strongly Disagree 
- NDA has received reversed data

NDA Formulae: 

```
shaps_total: sum all 14 items 
shaps_total_continuous: 56 – Total_score + 14
shaps_total_dichotomous: Count number of items < 3
```

In [ ]:
csvname = outdir / "shaps01.csv"
shaps_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
shaps_df["shaps_total"] = shaps_df[[f"shaps{v}" for v in range(1, 15)]].sum(axis=1)
shaps_df["shaps_total_continuous"] = 56 - shaps_df["shaps_total"] + 14
shaps_df["shaps_total_dichotomous"] = (shaps_df[[f"shaps{v}" for v in range(1,15)]] < 3).sum(axis=1)
shaps_df = shaps_df.convert_dtypes()
shaps_df.to_csv(csvname, index=False, na_rep=999)
display(shaps_df.head())
insert_dictname(csvname, "shaps", 1)

In [ ]:
shaps_df[["shaps_total", "shaps_total_continuous", "shaps_total_dichotomous"]].hist();

#### STAI

Adolescent NDA element formula for state and trait composites:

```
staiy_state: reverse(stai1) + reverse(stai2) + reverse(stai5) + reverse(stai_state8_i) + reverse(stai10) + reverse(stai11) + reverse(stai15) + reverse(stai16) + reverse(stai_state19_i) + reverse(stai20) + stai_state4_i + stai_state9_i + stai_state14_i + stai_state18_i + stai3 + stai6 + stai7 + stai12 + stai13 + stai17

staiy_trait: reverse(stai21) + reverse(stai26) + reverse(stai27) + reverse(stai30) + reverse(stai33) + reverse(stai36) + reverse(stai39) + stai_trait2_i + stai_trait3_i + stai_trait5_i + stai_trait11_i + stai_trait14_i + stai_trait15_i + stai24 + stai28 + stai29 + stai32 + stai37 + stai38 + stai40
```

Parent NDA element formula for state and trait composites: 

```
staiy_state: reverse(stai1) + reverse(stai2) + reverse(stai5) + reverse(stai8) + reverse(stai10) + reverse(stai11) + reverse(stai15) + reverse(stai16) + reverse(stai19) + reverse(stai20) + stai3 + stai4 + stai6 + stai7 + stai9 + stai12 + stai13 + stai14 + stai17 + stai18

staiy_trait: reverse(stai21) + reverse(stai23) + reverse(stai26) + reverse(stai27) + reverse(stai30) + reverse(stai33) + reverse(stai34) + reverse(stai36) + reverse(stai39) + stai22 + stai24 + stai25 + stai28 + stai29 + stai31 + stai32 + stai35 + stai37 + stai38 + stai40
```

In [ ]:
csvname = outdir / "stai01.csv"
stai_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
reverse_stai = lambda x: 5 - x
stai_df["staiy_state"] = (stai_df[["stai1", "stai2", "stai5", "stai_state8_i", "stai10", "stai11", "stai15", 
                                   "stai16", "stai_state19_i", "stai20"]].apply(reverse_stai).sum(axis=1) + \
                          stai_df[["stai_state4_i", "stai_state9_i", "stai_state14_i", "stai_state18_i", 
                                  "stai3", "stai6", "stai7", "stai12", "stai13", "stai17"]].sum(axis=1)) * \
                          (stai_df.respondent == "Child") + \
                         (stai_df[["stai1", "stai2", "stai5", "stai8", "stai10", "stai11", "stai15", 
                                   "stai16", "stai19", "stai20"]].apply(reverse_stai).sum(axis=1) + \
                          stai_df[["stai3", "stai4", "stai6", "stai7", "stai9", "stai12", "stai13", 
                                   "stai14", "stai17", "stai18"]].sum(axis=1)) * \
                          (stai_df.respondent == "Parent")
stai_df["staiy_trait"] = (stai_df[["stai21", "stai26", "stai27", "stai30", "stai33", "stai36", "stai39"]].apply(reverse_stai).sum(axis=1) + \
                          stai_df[["stai_trait2_i", "stai_trait3_i", "stai_trait5_i", "stai_trait11_i", "stai_trait14_i", "stai_trait15_i", 
                                   "stai24", "stai28", "stai29", "stai32", "stai37", "stai38", "stai40"]].sum(axis=1)) * \
                          (stai_df.respondent == "Child") + \
                         (stai_df[["stai21", "stai23", "stai26", "stai27", "stai30", "stai33", "stai34", 
                                   "stai36", "stai39"]].apply(reverse_stai).sum(axis=1) + \
                          stai_df[["stai22", "stai24", "stai25", "stai28", "stai29", "stai31", "stai32",
                                   "stai35", "stai37", "stai38", "stai40"]].sum(axis=1)) * \
                          (stai_df.respondent == "Parent")
stai_df = stai_df.convert_dtypes()
stai_df.to_csv(csvname, index=False, na_rep=999)
display(stai_df.head())
insert_dictname(csvname, "stai", 1)
stai_df[["respondent", "staiy_state"]].groupby("respondent").hist();
stai_df[["respondent", "staiy_trait"]].groupby("respondent").hist();

#### CBCL 

**There are several other raw scores that are required in the NDA that do not appear below**

Ref: Achenbach, 2001© CBCL/6-18 Syndrome Scales  

8 syndrome scales:

```
cbcl_anxious_raw: [14, 29, 30, 31, 32, 33, 35, 45, 50, 52, 71, 91, 112]
cbcl_withdrawn_raw: [5, 42, 65, 69, 75, 102, 103, 111]
cbcl_somatic_c_raw: [47, 49, 51, 54, 56a-g]
cbcl_social_p_raw: [11, 12, 25, 27, 34, 36, 38, 48, 62, 64, 79]
cbcl_thought_raw: [9, 18, 40, 46, 58, 59, 60, 66, 70, 76, 83, 84, 85, 92, 100]
cbcl_attention_raw: [1, 4, 8, 10, 13, 17, 41, 61, 78, 80]
cbcl_rulebreak_raw: [2, 26, 28, 39, 43, 63, 67, 72, 73, 81, 82, 90, 96, 99, 101, 105, 106]
cbcl_aggresive_raw: [3, 16, 19, 20, 21, 22, 23, 37, 57, 68, 86, 87, 88, 89, 94, 95, 97, 104]

(NOT IN NDA) 
cbcl_other_raw: [6, 7, 15, 24, 44, 53, 55, 56h, 74, 77, 93, 98, 107, 108, 109, 110, 113]
```

Broader problem scales: 

```
cbcl_internal_raw: [anxious_depressed, withdrawn_depressed, somatic_complaints]
cbcl_external_raw: [rule_breaking, aggressive_behavior]
cbcl_total_raw: sum ALL 8 syndrome scale totals + cbcl_other_raw items detailed above 
```

Cbcl/6-18 DSM-Oriented Scales (6 scales) 

```
cbcl_deresspr_raw: [5, 14, 18, 24, 35, 52, 54, 76, 77, 91, 100, 102, 103]
cbcl_anxiety_raw: [11, 29, 30, 31, 45, 47, 50, 71, 112]
cbcl_somatic_p_raw: [56a-g]
cbcl_adhd_raw: [4, 8, 10, 41, 78, 93, 104]
cbcl_oppositional_raw: [3, 22, 23, 86, 95]
cbcl_conduct_raw: [15, 16, 21, 26, 28, 37, 39, 43, 57, 67, 72, 81, 82, 90, 97, 101, 106]
```

In [ ]:
csvname = outdir / "cbcl01.csv"
cbcl_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
cbcl_dict = dict(cbcl_anxious_raw=[14, 29, 30, 31, 32, 33, 35, 45, 50, 52, 71, 91, 112],
                 cbcl_withdrawn_raw=[5, 42, 65, 69, 75, 102, 103, 111],
                 cbcl_somatic_c_raw=[47, 49, 51, 54, "56a", "56b", "56c", "56d", "56e", "56f", "56g"],
                 cbcl_social_p_raw=[11, 12, 25, 27, 34, 36, 38, 48, 62, 64, 79],
                 cbcl_thought_raw=[9, 18, 40, 46, 58, 59, 60, 66, 70, 76, 83, 84, 85, 92, 100],
                 cbcl_attention_raw=[1, 4, 8, 10, 13, 17, 41, 61, 78, 80],
                 cbcl_rulebreak_raw=[2, 26, 28, 39, 43, 63, 67, 72, 73, 81, 82, 90, 96, 99, 101, 105, 106],
                 cbcl_aggressive_raw=[3, 16, 19, 20, 21, 22, 23, 37, 57, 68, 86, 87, 88, 89, 94, 95, 97, 104],
                 cbcl_other_raw=[6, 7, 15, 24, 44, 53, 55, "56h", 74, 77, 93, 98, 107, 108, 109, 110, "113a"],
                 cbcl_depresspr_raw=[5, 14, 18, 24, 35, 52, 54, 76, 77, 91, 100, 102, 103],
                 cbcl_anxiety_raw=[11, 29, 30, 31, 45, 47, 50, 71, 112],
                 cbcl_somatic_p_raw=["56a", "56b", "56c", "56d", "56e", "56f", "56g"],
                 cbcl_adhd_raw=[4, 8, 10, 41, 78, 93, 104],
                 cbcl_oppositional_raw=[3, 22, 23, 86, 95],
                 cbcl_conduct_raw=[15, 16, 21, 26, 28, 37, 39, 43, 57, 67, 72, 81, 82, 90, 97, 101, 106]
                )
for key, val in cbcl_dict.items():
    columns = [f"cbcl{v}" for v in val]
    cbcl_df[key] = cbcl_df[columns].sum(axis=1)
cbcl_df["cbcl_internal_raw"] = cbcl_df[["cbcl_anxious_raw", "cbcl_withdrawn_raw", "cbcl_somatic_c_raw"]].sum(axis=1)
cbcl_df["cbcl_external_raw"] = cbcl_df[["cbcl_rulebreak_raw", "cbcl_aggressive_raw"]].sum(axis=1)
cbcl_df["cbcl_total_raw"] = cbcl_df[["cbcl_internal_raw", 
                                     "cbcl_social_p_raw", "cbcl_thought_raw", "cbcl_attention_raw",
                                     "cbcl_external_raw", 
                                     "cbcl_other_raw"]].sum(axis=1)
cbcl_df = cbcl_df.drop(["cbcl_other_raw"], axis=1)
cbcl_df = cbcl_df.convert_dtypes()
cbcl_df.to_csv(csvname, index=False, na_rep=999)
display(cbcl_df.head())
insert_dictname(csvname, "cbcl", 1)

#### MASQ

From Watson and Clark, 1991 62-item Short-form Scoring Key. See also Watson et al., 1995 J Abnormal Psychology 

masq2_score_aa (Anxious arousal subtest) 
masq2_score_ad (anhedonic depression subtest) 
masq2_score_gd (general distress anxious symptoms subtest) 
masq2_score_gdd (general distress depressive symptoms subtest) 

```
masq2_score_aa: [2, 6, 13, 17, 19, 24, 28, 30, 37, 40, 42, 44, 46, 48, 52, 54, 62]
masq2_score_ad: 

- REVERSE: 3, 7, 10, 15, 22, 27, 39, 43, 47, 49, 53, 56, 58, 60  
- Normal: 18, 25, 33, 41, 50, 51, 57, 61 

masq2_score_gd: [4, 8, 11, 14, 16, 20, 26, 32, 35, 55, 59]
masq2_score_gdd: [1, 5, 9, 12, 21, 23, 29, 31, 34, 36, 38, 45]
```

```
ANHEDONIC DEPRESSION
AD= SYMP18 + SYMP25 + SYMP33 + SYMP41 + SYMPSO + SYMP51 +
SYMP57 + SYMP61 + 84- (SYMP3 + SYMP7 + SYMPlO + SYMP15 +
SYMP22 + SYMP27 + SYMP39 + SYMP43 + SYMP47 + SYMP49 + SYMP53 +
SYMP56 + SYMP58 + SYMP60);
```

Note: scl14 is a string in NDA. The MASQ structure does not directly conform to the 62 item scale. 

For the masq2_score_ad composite, it:

1. Sums all non-reverse scored items (Sum A) and then adds 84 
1. For reverse scored items it takes 6 - item score, then sums these (Sum B)
1. Then it subtracts Sum B from Sum A ([Sum A + 84] - Sum B)

MASQ required a second level of remapping since the names in NDA differ
significantly from the names in the MASQ


In [ ]:
banda2nda_masq_map = pd.read_csv("masq_summary_key.csv")
banda2nda_masq_map.head()

In [ ]:
csvname = outdir / "masq01.csv"
masq_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
masq_dict = dict(masq2_score_aa=[2, 6, 13, 17, 19, 24, 28, 30, 37, 40, 42, 44, 46, 48, 52, 54, 62],
                 masq2_score_gd=[4, 8, 11, 14, 16, 20, 26, 32, 35, 55, 59],
                 masq2_score_gdd=[1, 5, 9, 12, 21, 23, 29, 31, 34, 36, 38, 45]
                )
banda2nda_masq = dict(zip(banda2nda_masq_map.banda_var, banda2nda_masq_map.ndar_var))
nda_masq_items = list(banda2nda_masq_map.ndar_var)
recoded_df = masq_df[["subjectkey", "src_subject_id","visit"] + nda_masq_items].copy()
masq_func_map = {'None': lambda x: x,
                 'add 1': lambda x: x + 1,
                 'add 1, reverse': lambda x: 6 - (x + 1), 
                 'reverse': lambda x: 6 - x,
                 'convert from string': lambda x: {"not at all": 1, "a little bit": 2, "moderately": 3,
                                                     "quite a bit": 4, "extremely": 5}[x]
                }

ndar_masq_xfm = dict(zip(banda2nda_masq_map.ndar_var, banda2nda_masq_map["Transform for subscale within NDA"]))
for key in nda_masq_items:
    recoded_df[key] = recoded_df[key].apply(masq_func_map[ndar_masq_xfm[key]])
for key, val in masq_dict.items():
    columns = [banda2nda_masq[f"masq{v}"] for v in val]
    masq_df[key] = recoded_df[columns].sum(axis=1)
masq2_score_ad_reverse = [3, 7, 10, 15, 22, 27, 39, 43, 47, 49, 53, 56, 58, 60]
masq2_score_ad_normal = [18, 25, 33, 41, 50, 51, 57, 61]
masq_df["masq2_score_ad"] = recoded_df[[banda2nda_masq[f"masq{v}"] for v in masq2_score_ad_normal]].sum(axis=1) + 84 \
                          - recoded_df[[banda2nda_masq[f"masq{v}"] for v in masq2_score_ad_reverse]].sum(axis=1)
masq_df = masq_df.convert_dtypes()
masq_df.to_csv(csvname, index=False, na_rep=999)
display(masq_df.head())
insert_dictname(csvname, "masq", 1)

#### RMBI

see Gladstone and Parker, 2005 for scores 

- Orig Scale: 0 No/Hardly Ever – 2 Yes/Most of Time 
- Reverse Scale: 2 No/Hardly Ever – 0 Yes/Most of Time 

reverse score items: 4, 5, 7, 11, 13, 15 

For NDA: 

- For normal score items replace 3’s with 0 
- For reverse score items use: max(0, item – 1) 

```
rmbi_fi (fearful inhibition): 2, 4, 5, 9, 11, 15 
rmbi_na (non-approach): 1, 6, 10, 16, 18 
rmbi_ra (risk avoidance): 7, 8, 13 
rmbi_ss (shyness and sensitivity): 3, 12, 14, 17 
```

In [ ]:
csvname = outdir / "rmbi01.csv"
rmbi_df = pd.read_csv(csvname, na_values=[999], skiprows=[0])
rmbi_dict = dict(rmbi_fi=[2, 4, 5, 9, 11, 15],
                 rmbi_na=[1, 6, 10, 16, 18],
                 rmbi_ra=[7, 8, 13],
                 rmbi_ss=[3, 12, 14, 17],
                 rmbi_total=list(range(1, 19))
                )
rmbi_items = [f"rmbi{v}" for v in range(1, 19)]
recoded_df = rmbi_df[["subjectkey", "src_subject_id","visit"] + rmbi_items].copy()
reverse_items = [f"rmbi{v}" for v in [4, 5, 7, 11, 13, 15]]
recoded_df[reverse_items] = (recoded_df[reverse_items] - 1).copy().clip(lower=0)
recoded_df[rmbi_items] = recoded_df[rmbi_items].applymap(lambda x: 0 if x == 3 else x)
for key, val in rmbi_dict.items():
    columns = [f"rmbi{v}" for v in val]
    rmbi_df[key] = recoded_df[columns].sum(axis=1)
rmbi_df = rmbi_df.convert_dtypes()
rmbi_df.to_csv(csvname, index=False, na_rep=999)
display(rmbi_df.head())
insert_dictname(csvname, "rmbi", 1)

### HACK/FIX NDA submission ISSUES

In addition to the recoding needed to convert NDA data elements to relevant 
items for composite scoring described above, NDA often requires specific types
of values to encode missing data. 

This section applies these NDA structure specific elements to the data

In [ ]:
for fname in ["pwmt01", "pmat01"]:
    csvname = outdir / (fname + ".csv")
    df_name = pd.read_csv(csvname, skiprows=[0])
    df_name.valid_code = df_name.valid_code.apply(lambda x: 'F' if x=="999" or x=="0" else x)
    df_name = df_name.convert_dtypes()
    df_name.to_csv(csvname, index=False, na_rep='')
    insert_dictname(csvname, fname[:-2], int(fname[-2:]))

fname = "demographics02"
csvname = outdir / (fname + ".csv")
df_name = pd.read_csv(csvname, na_values=[999, "999", ""], skiprows=[0])
df_name.demo_other_parent_educ = df_name.demo_other_parent_educ.apply(lambda x: np.nan if x==9 else x)
df_name = df_name.convert_dtypes()
df_name.to_csv(csvname, index=False, na_rep='')
insert_dictname(csvname, fname[:-2], int(fname[-2:]))

# 999 to empty
affected = ["er4001", "masq01", "mfq01", "nffi01", "pmat01", "rbqa01", "rcads01", "rmbi01",
            "shaps01", "cssrs01", "stai01", "pwmt01", "fhs01", "chaphand01"]

for fname in affected:
    csvname = outdir / (fname + ".csv")
    df_name = pd.read_csv(csvname, na_values=[999, "999", ""], skiprows=[0])
    if fname == "shaps01":
        df_name.shaps_total = df_name.shaps_total.apply(lambda x: 999 if np.isnan(x) else x)
    if fname == "masq01":
        for field in ["masq2_score_aa", "masq2_score_ad", "masq2_score_gd", "masq2_score_gdd"]:
            df_name[field] = df_name[field].apply(lambda x: 999 if np.isnan(x) else x)
    df_name = df_name.convert_dtypes()
    df_name.to_csv(csvname, index=False, na_rep='')
    insert_dictname(csvname, fname[:-2], int(fname[-2:]))



for fname, field in [("lswmt01", "nihtbx_list_fullycorrected"), 
                     ("pcps01", "nihtbx_pattern_fullycorrected"),
                     ("dccs01", None)]:
    csvname = outdir / (fname + ".csv")
    df_name = pd.read_csv(csvname, na_values=[999, "999", ""], skiprows=[0])
    if field is not None:
        df_name[field] = df_name[field].apply(lambda x: 0 if np.isnan(x) else x)
        na_rep = ''
    else:
        na_rep = 0
    df_name = df_name.convert_dtypes()
    df_name.to_csv(csvname, index=False, na_rep=na_rep)
    insert_dictname(csvname, fname[:-2], int(fname[-2:]))

fname = "tanner_sms01"
csvname = outdir / (fname + ".csv")
df_name = pd.read_csv(csvname, na_values=[999, "999", ""], skiprows=[0])
df_name.tsmfemale = df_name.tsmfemale.apply(lambda x: 9999 if np.isnan(x) else x)
df_name.tsmmale = df_name.tsmmale.apply(lambda x: 9999 if np.isnan(x) else x)
df_name.tsmboth = df_name.tsmboth.apply(lambda x: 9998 if np.isnan(x) else x)
df_name = df_name.convert_dtypes()
df_name.to_csv(csvname, index=False, na_rep='')
insert_dictname(csvname, fname[:-2], int(fname[-2:]))

fname = "cbcl01"
csvname = outdir / (fname + ".csv")
df_name = pd.read_csv(csvname, skiprows=[0])
pr_cols = [var for var in df_name.columns if "cbcl_pr" in var]
df_name = df_name.drop(pr_cols, axis=1)
df_name = df_name.convert_dtypes()
df_name.to_csv(csvname, index=False)
insert_dictname(csvname, fname[:-2], int(fname[-2:]))

### Create missingness dataframe

This indicates which assessments were missing for each subject at
the different visits.

In [ ]:
outdir = Path("output-2022-10-27 01:11:31.202796")
outcsvs = sorted(glob(str(outdir / "*.csv")))
len(outcsvs)

In [ ]:
df_missing = pd.DataFrame()
fields = ["subjectkey", "src_subject_id", "visit"]
for csv in outcsvs:
    df_new = pd.read_csv(csv, skiprows=[0])
    structname = Path(csv).name.split(".csv")[0]
    indices = [tuple(val) for val in df_new[fields].values.tolist()]
    indices = pd.Index(indices).drop_duplicates()
    indices.names = fields
    df_missing = pd.concat((df_missing, 
                            pd.DataFrame(False, columns=[structname],
                                         index=indices)),
                            axis=1)


In [ ]:
df_missing

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.facecolor':'white'})

In [ ]:
for val in df_missing.groupby("visit"):
    plt.figure(figsize=(10, 10*val[1].shape[0] / val[1].shape[1]))
    hp = sns.heatmap(val[1].droplevel("visit").fillna(True).sort_index(level="src_subject_id"),
                     cmap="binary", linewidths=0.1,linecolor="gray",
                     cbar=None)
    plt.title(f"visit: {val[0]}")
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.savefig(f"{val[0]}.png", bbox_inches='tight')

In [ ]:
(df_missing.fillna(True) == False).sort_index(level=["visit", "src_subject_id"]).reset_index().to_csv("Completeness.csv", index=False)